In [40]:
import os
import json
import numpy as np
from glob import glob
from PIL import Image
from dataclasses import dataclass

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

DATA_ROOT_PATH = "../eye_tracker_auto_labeller/data"

FACE_OVAL_LANDMARK_IDX_LIST = [
    162,  21,  54, 103,  67, 109,  10, 338, 297, 332, 284, 251, 389,
    356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148,
    176, 149, 150, 136, 172,  58, 132,  93, 234, 127, 162,
]
LEFT_EYE_LANDMARK_IDX_LIST = [
    263, 249, 390, 373, 374, 380, 381, 382, 362, 
    398, 384, 385, 386, 387, 388, 466, 263
]
LEFT_IRIS_LANDMARK_IDX_LIST = [475, 476, 477, 474, 475]
RIGHT_EYE_LANDMARK_IDX_LIST = [
     33,   7, 163, 144, 145, 153, 154, 155, 133,
    173, 157, 158, 159, 160, 161, 246, 33
]
RIGHT_IRIS_LANDMARK_IDX_LIST = [471, 472, 469, 470, 471]

OVAL_UPPER_HALF = [361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172,  58, 132]
OVAL_LOWER_HALF = [234, 127, 162,  21,  54, 103,  67, 109,  10, 338, 297, 332, 284, 251, 389, 356, 454]
OVAL_LEFT_HALF  = [338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377]
OVAL_RIGHT_HALF = [148, 176, 149, 150, 136, 172,  58, 132,  93, 234, 127, 162,  21,  54, 103,  67, 109]

class EyeTrackerDataset(Dataset) :
    def __init__(
        self,
        data_root_path = DATA_ROOT_PATH,
        face_oval_landmark_idx_list     = FACE_OVAL_LANDMARK_IDX_LIST,
        left_eye_landmark_idx_list      = LEFT_EYE_LANDMARK_IDX_LIST,
        left_iris_landmark_idx_list     = LEFT_IRIS_LANDMARK_IDX_LIST,
        right_eye_landmark_idx_list     = RIGHT_EYE_LANDMARK_IDX_LIST,
        right_iris_landmark_idx_list    = RIGHT_IRIS_LANDMARK_IDX_LIST,
        return_landmark = True,
        return_image = False
    ) :
        super(EyeTrackerDataset, self).__init__()
        self.DATA_ROOT_PATH = data_root_path
        self.face_oval_landmark_idx_list    = face_oval_landmark_idx_list
        self.left_eye_landmark_idx_list     = left_eye_landmark_idx_list
        self.left_iris_landmark_idx_list    = left_iris_landmark_idx_list
        self.right_eye_landmark_idx_list    = right_eye_landmark_idx_list
        self.right_iris_landmark_idx_list   = right_iris_landmark_idx_list
        self.return_landmark = return_landmark
        self.return_image = return_image

        self.label_path_list = sorted(glob(
            f"{self.DATA_ROOT_PATH}/*/*.json"
        ))
        self.image_path_list = sorted(glob(
            f"{self.DATA_ROOT_PATH}/*/*.png"
        ))

        # there might exists case where eiter image or label does not exist
        # ex) a.json exist but a.png doesn't exist or vice-versa.
        # filter this cases to ensure every data are image-label pair.
        self.label_path_list = list(filter(
            lambda file_name : file_name.replace("json", "png") in self.image_path_list,
            self.label_path_list
        ))
        self.image_path_list = list(filter(
            lambda file_name : file_name.replace("png", "json") in self.label_path_list,
            self.image_path_list
        ))
        assert len(self.label_path_list) == len(self.image_path_list)

        self.to_torch = transforms.ToTensor()

    def __len__(self) :
        return len(self.label_path_list)
    
    def __getitem__(self, index):
        with open(self.label_path_list[index], "r") as fp :
            label_data = json.load(fp)
        mouse_position = torch.Tensor(label_data["mouse_position"])
        data = {
            "mouse_position" : mouse_position
        }
        if self.return_landmark :
            face_landmark_array = torch.Tensor(label_data["face_landmark_array"])
            face_oval_landmark_array  = face_landmark_array[self.face_oval_landmark_idx_list]
            left_eye_landmark_array   = face_landmark_array[self.left_eye_landmark_idx_list]
            right_eye_landmark_array  = face_landmark_array[self.right_eye_landmark_idx_list]
            left_iris_landmark_array  = face_landmark_array[self.left_iris_landmark_idx_list]
            right_iris_landmark_array = face_landmark_array[self.right_iris_landmark_idx_list]

            data["face_landmark_array"] = face_landmark_array
            data["face_oval_landmark_array"] = face_oval_landmark_array
            data["left_eye_landmark_array"] = left_eye_landmark_array
            data["right_eye_landmark_array"] = right_eye_landmark_array
            data["left_iris_landmark_array"] = left_iris_landmark_array
            data["right_iris_landmark_array"] = right_iris_landmark_array
            data["face_oval_mean"] = face_oval_landmark_array.mean(axis=0)

            eye_center_pos = face_landmark_array[LEFT_EYE_LANDMARK_IDX_LIST[0] + RIGHT_EYE_LANDMARK_IDX_LIST[0]].mean(axis=0)
            data["eye_center_pos"] = eye_center_pos
            
            face_dir_x = torch.mean(face_landmark_array[OVAL_RIGHT_HALF], axis=0) - torch.mean(face_landmark_array[OVAL_LEFT_HALF],  axis=0)
            face_dir_y = torch.mean(face_landmark_array[OVAL_LOWER_HALF], axis=0) - torch.mean(face_landmark_array[OVAL_UPPER_HALF], axis=0)
            face_dir_z = torch.cross(face_dir_x, face_dir_y)
            face_area_factor = torch.sqrt(torch.sqrt(torch.sum(torch.square(face_dir_z))))
            data["face_area_factor"] = face_area_factor

            face_dir_x_norm = face_dir_x / torch.sqrt(torch.sum(face_dir_x * face_dir_x))
            face_dir_y_norm = face_dir_y / torch.sqrt(torch.sum(face_dir_y * face_dir_y))
            face_dir_z_norm = face_dir_z / torch.sqrt(torch.sum(face_dir_z * face_dir_z))
            data["face_dir_x_norm"] = face_dir_x_norm
            data["face_dir_y_norm"] = face_dir_y_norm
            data["face_dir_z_norm"] = face_dir_z_norm

            pitch = torch.arctan2(-face_dir_z_norm[1], -face_dir_z_norm[2])
            yaw   = torch.arcsin(face_dir_z_norm[0])
            roll  = torch.arctan2(-face_dir_y_norm[0], face_dir_x_norm[0])
            data["roll"]  = roll
            data["pitch"] = pitch
            data["yaw"]   = yaw

        if self.return_image :
            image = Image.open(self.image_path_list[index])
            left_eye_lt_rb = np.array([
                left_eye_landmark_array.numpy().min(axis=0),
                left_eye_landmark_array.numpy().max(axis=0),
            ])[:, :2] * np.array([image.width, image.height])
            #left_eye_lt_rb = left_eye_lt_rb * 2 - left_eye_lt_rb.mean(axis=0)
            left_eye_lt_rb = left_eye_lt_rb.flatten().astype(int)

            right_eye_lt_rb = np.array([
                right_eye_landmark_array.numpy().min(axis=0),
                right_eye_landmark_array.numpy().max(axis=0),
            ])[:, :2] * np.array([image.width, image.height])
            #right_eye_lt_rb = right_eye_lt_rb * 2 - right_eye_lt_rb.mean(axis=0)
            right_eye_lt_rb = right_eye_lt_rb.flatten().astype(int)

            left_eye_image  = self.to_torch(image.crop(left_eye_lt_rb))
            right_eye_image = self.to_torch(image.crop(right_eye_lt_rb))
            
            data["image"] =  self.to_torch(image)
            data["left_eye_image"] =  left_eye_image
            data["right_eye_image"] =  right_eye_image
        
        return data

In [43]:
eye_tracker_dataset = EyeTrackerDataset(DATA_ROOT_PATH, return_image=True)

data = eye_tracker_dataset[1600]

data.keys()

dict_keys(['mouse_position', 'face_landmark_array', 'face_oval_landmark_array', 'left_eye_landmark_array', 'right_eye_landmark_array', 'left_iris_landmark_array', 'right_iris_landmark_array', 'face_oval_mean', 'eye_center_pos', 'face_area_factor', 'face_dir_x_norm', 'face_dir_y_norm', 'face_dir_z_norm', 'roll', 'pitch', 'yaw', 'image', 'left_eye_image', 'right_eye_image'])

In [ ]:
def positional_encoding(val, dim=3) :
     